In [1]:
import os

In [2]:
%pwd

'/home/kalema/Projects/ML/Wine-Quality-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/kalema/Projects/ML/Wine-Quality-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    l1_ratio: float
    alpha: float
    model_name: str
    target_column: str

In [7]:
from wineQuality.constants import *
from wineQuality.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            l1_ratio=params.l1_ratio,
            alpha=params.alpha,
            target_column=schema.name
        )

        return model_training_config

In [9]:
import pandas as pd
import os
from wineQuality import logger
from sklearn.linear_model import ElasticNet
import joblib

[2024-03-30 02:47:08,434: INFO: utils: NumExpr defaulting to 4 threads.]


In [12]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    model_training.train()
except Exception as e:
    raise e

[2024-03-30 03:08:35,225: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-30 03:08:35,232: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-30 03:08:35,238: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-30 03:08:35,248: INFO: common: Created directory at: artifacts/model_training]
